In [1]:
#Example script. Solve one HII region with nH=100 cm^-3, Z=0.1Z_sun, T4=1.
#Assume the incident stellar radiation spectrum shape is given by Conroy et al. (2010), with Chabrier IMF.
#Assume stellar population age of 6 Myr, metallicity of 0.1Z_sun.
#Assume the incident spectrum hydrogen ionizing photon generation rate is 10^50 s^-1.
from __future__ import division
import h5py
import numpy as np
from HIILines import *
import matplotlib.pyplot as plt

#Load incident spectrum. Replace this with your own spectrum.
#The spectrum lookup table can be downloaded from https://zenodo.org/record/6338462#.ZBENinbMKbh
fSEDPath     = 'tables/SSP_Spectra_Conroy-et-al_v2.5_imfChabrier.hdf5'
fSEDTable    = h5py.File(fSEDPath,'r')
ageGrid      = fSEDTable['ages'][:]                #Gyr
logZstarGrid = fSEDTable['metallicities'][:]       #log10(Z_sun)
Spec         = fSEDTable['spectra'][:]             #L_sun/Hz
wl           = fSEDTable['wavelengths'][:]         #A
nu           = c*10**13/wl                         #Hz
ageIdx       = 26                                  #Set the stellar age identical to the 26th grid. ~6Myr
logZstarIdx  = 9                                   #Set the stellar metallicity identical to the 9th grid. ~0.1 Z_sun
L            = Spec[logZstarIdx,ageIdx,::-1]*Lsun2Jps #J/s/Hz
nu           = nu[::-1]

#Input parameters
logZgas      = -1                                  #log10(Z_sun)
logQ         = 50                                  #log10(s^-1)
lognH        = 2                                   #log10(cm^-3)
T4OII        = 1                                   #OII region temperature (10000K)
T4OIII       = 1                                   #OIII region temperature (10000K)

#Solve for OIII and OII region volumes
VOII2VHII_TOII, VOIII2VHII_TOII   = solV(nu,L,logQ,lognH,logZgas,T4OII)
VOII2VHII_TOIII, VOIII2VHII_TOIII = solV(nu,L,logQ,lognH,logZgas,T4OIII)
VOII2VHII      = VOII2VHII_TOII/(VOII2VHII_TOII+VOIII2VHII_TOIII)
VOIII2VHII     = VOIII2VHII_TOIII/(VOII2VHII_TOII+VOIII2VHII_TOIII)
#Compute HII region line luminosities
INPUT        = [logQ, lognH, logZgas, T4OII, T4OIII, VOII2VHII, VOIII2VHII]
HIILineL     = assignL(INPUT)

print('HIILines finishes solving this HII region.')
print('log10(OII 3727,29A luminosity/[L_sun]) is:'  ,np.log10(10**HIILineL[0]+10**HIILineL[1]))
print('log10(OIII 88 micron luminosity/[L_sun]) is:',HIILineL[6])
print('log10(OIII 52 micron luminosity/[L_sun]) is:',HIILineL[7])
print('log10(OIII 4960.3A luminosity/[L_sun]) is:'  ,HIILineL[8])
print('log10(OIII 5008.2A luminosity/[L_sun]) is:'  ,HIILineL[9])
print('log10(OIII 4364.4A luminosity/[L_sun]) is:'  ,HIILineL[10])
print('log10(H_beta luminosity/[L_sun]) is:'        ,HIILineL[12])

HIILines finishes solving this HII region.
log10(OII 3727,29A luminosity/[L_sun]) is: 4.003380859942073
log10(OIII 88 micron luminosity/[L_sun]) is: 3.319079727090426
log10(OIII 52 micron luminosity/[L_sun]) is: 3.236352409256178
log10(OIII 4960.3A luminosity/[L_sun]) is: 3.345271083873826
log10(OIII 5008.2A luminosity/[L_sun]) is: 3.81204957937095
log10(OIII 4364.4A luminosity/[L_sun]) is: 1.6550764689576245
log10(H_beta luminosity/[L_sun]) is: 4.096649385303536


In [2]:
#Input parameters
agestar      = ageGrid[ageIdx]*1000                #~6 Myr
logZstar     = logZstarGrid[logZstarIdx]           #log10(Z_sun)~-1
logZgas      = -1                                  #log10(Z_sun)
logQ         = 50                                  #log10(s^-1)
lognH        = 2                                   #log10(cm^-3)
T4OII        = 1                                   #OII region temperature (10000K)
T4OIII       = 1                                   #OIII region temperature (10000K)

#Load VOII/VHII lookup table
#The VOII/VHII lookup table can be downloaded from zenedo (https://zenodo.org/record/8213031).
VOII2VHIITable = h5py.File('tables/VOII2VHII.hdf5')
ageGrid        = VOII2VHIITable['ages'][:]
logZstarGrid   = VOII2VHIITable['metallicities'][:]
lognHGrid      = VOII2VHIITable['lognH'][:]
logQGrid       = VOII2VHIITable['logQ'][:]
logZismGrid    = VOII2VHIITable['logZ'][:]
T4Grid         = VOII2VHIITable['T4'][:]
VOII2VHII      = VOII2VHIITable['VOII2VHII'][:]
VOIII2VHII     = VOII2VHIITable['VOIII2VHII'][:]
VOII2VHIITable.close()

VOII2VHIIf     = interpolate.RegularGridInterpolator((logQGrid,lognHGrid,logZismGrid,T4Grid,logZstarGrid,ageGrid*1000),VOII2VHII)
VOIII2VHIIf    = interpolate.RegularGridInterpolator((logQGrid,lognHGrid,logZismGrid,T4Grid,logZstarGrid,ageGrid*1000),VOIII2VHII)

INPUT            = [logQ, lognH, logZgas, T4OII, logZstar, agestar]
VOII2VHII_TOII   = VOII2VHIIf(INPUT)[0]
INPUT            = [logQ, lognH, logZgas, T4OIII, logZstar, agestar]
VOIII2VHII_TOIII = VOIII2VHIIf(INPUT)[0]
VOII2VHII        = VOII2VHII_TOII/(VOII2VHII_TOII+VOIII2VHII_TOIII)
VOIII2VHII       = VOIII2VHII_TOIII/(VOII2VHII_TOII+VOIII2VHII_TOIII)
#Compute HII region line luminosities
INPUT        = [logQ, lognH, logZgas, T4OII, T4OIII, VOII2VHII, VOIII2VHII]
HIILineL     = assignL(INPUT)

print('HIILines finishes solving this HII region.')
print('log10(OII 3727,29A luminosity/[L_sun]) is:'  ,np.log10(10**HIILineL[0]+10**HIILineL[1]))
print('log10(OIII 88 micron luminosity/[L_sun]) is:',HIILineL[6])
print('log10(OIII 52 micron luminosity/[L_sun]) is:',HIILineL[7])
print('log10(OIII 4960.3A luminosity/[L_sun]) is:'  ,HIILineL[8])
print('log10(OIII 5008.2A luminosity/[L_sun]) is:'  ,HIILineL[9])
print('log10(OIII 4364.4A luminosity/[L_sun]) is:'  ,HIILineL[10])
print('log10(H_beta luminosity/[L_sun]) is:'        ,HIILineL[12])

HIILines finishes solving this HII region.
log10(OII 3727,29A luminosity/[L_sun]) is: 4.003380859942073
log10(OIII 88 micron luminosity/[L_sun]) is: 3.319079727090426
log10(OIII 52 micron luminosity/[L_sun]) is: 3.236352409256178
log10(OIII 4960.3A luminosity/[L_sun]) is: 3.345271083873826
log10(OIII 5008.2A luminosity/[L_sun]) is: 3.81204957937095
log10(OIII 4364.4A luminosity/[L_sun]) is: 1.6550764689576245
log10(H_beta luminosity/[L_sun]) is: 4.096649385303536
